In [114]:
N = 10

list1 = rand(N,1)
list2 = rand(N,1);

In [115]:
r_tuple = [(list1[i],list2[i]) for i = 1:length(list1)];

In [123]:
newm_bench = zeros(length(r_tuple),2)
@time for i = 1:length(r_tuple)
    newm_bench[i,:] = [r_tuple[i][1],r_tuple[i][2]]
end

  0.000184 seconds (71 allocations: 2.375 KiB)


In [124]:
@time newm = reshape(reinterpret(Float64, r_tuple),(:,2));

  0.000018 seconds (2 allocations: 80 bytes)


In [128]:
newm

10×2 reshape(reinterpret(Float64, ::Vector{Tuple{Float64, Float64}}), 10, 2) with eltype Float64:
 0.709458   0.730812
 0.732603   0.175381
 0.147133   0.970691
 0.325354   0.950692
 0.0836828  0.186869
 0.894911   0.828769
 0.778858   0.189057
 0.248745   0.401042
 0.690266   0.388252
 0.194431   0.570441

In [121]:
newm

10×2 reshape(reinterpret(Float64, ::Vector{Tuple{Float64, Float64}}), 10, 2) with eltype Float64:
 0.709458   0.730812
 0.732603   0.175381
 0.147133   0.970691
 0.325354   0.950692
 0.0836828  0.186869
 0.894911   0.828769
 0.778858   0.189057
 0.248745   0.401042
 0.690266   0.388252
 0.194431   0.570441

In [137]:
A,B = findmax(newm)

(0.970690562643393, CartesianIndex(3, 2))

In [141]:
[BB1,BB2] = B

LoadError: syntax: invalid assignment location "[BB1, BB2]" around In[141]:1

In [135]:
B[3]

LoadError: BoundsError: attempt to access Tuple{Int64, Int64} at index [3]

In [37]:
r_tuple[:,2]

LoadError: BoundsError: attempt to access 10-element Vector{Tuple{Float64, Float64}} at index [1:10, 2]

In [144]:
[0.2
    ,0.5]

2-element Vector{Float64}:
 0.2
 0.5

In [145]:
function XHloop_append(i,H_film_start_one,H_film_end_one,sys)
    Xp = sys.liquid.Xp[i]
    δstart = sys.vapor.δstart[i]
    δend = sys.vapor.δend[i]
    Lfilm_start = sys.vapor.Lfilm_start[i]
    Lfilm_end = sys.vapor.Lfilm_end[i]
    
    Hvapor = sys.vapor.Hᵥ
    
    L = sys.tube.L
    
    if Xp[2] > Xp[1]
        
        return "Xp error 1"
        
    elseif Xp[1] + Lfilm_start > L
        
        X_inner_loop_append = mod.([sys.vapor.Xp[i][1],L,
                0.0,sys.vapor.Xp[i][1]+Lfilm_start,
                sys.vapor.Xp[i][1]+Lfilm_start,sys.vapor.Xp[i][end]-Lfilm_end,
                sys.vapor.Xp[i][end]-Lfilm_end,sys.vapor.Xp[i][end]],L)
        
        H_inner_loop_append = [H_film_start[i], H_film_start[i],
                H_film_start[i], H_film_start[i],
                H_vapor[i],H_vapor[i],
                H_film_end[i],H_film_end[i]]
        
    elseif Xp[2] - Lfilm_end > L
        
        X_inner_loop_append = mod.([sys.vapor.Xp[i][1],sys.vapor.Xp[i][1]+Lfilm_start,
                sys.vapor.Xp[i][1]+Lfilm_start, L,
                0.0, sys.vapor.Xp[i][end]-Lfilm_end,
                sys.vapor.Xp[i][end]-Lfilm_end,sys.vapor.Xp[i][end]],L)
        
        H_inner_loop_append = [H_film_start[i], H_film_start[i],
                H_vapor[i], H_vapor[i],
                H_vapor[i],H_vapor[i],
                H_film_end[i],H_film_end[i]]
        
    elseif Xp[2] > L
        
        X_inner_loop_append = mod.([sys.vapor.Xp[i][1],sys.vapor.Xp[i][1]+Lfilm_start,
                sys.vapor.Xp[i][1]+Lfilm_start, L,
                0.0, sys.vapor.Xp[i][end]-Lfilm_end,
                sys.vapor.Xp[i][end]-Lfilm_end,sys.vapor.Xp[i][end]],L)
        
        H_inner_loop_append = [H_film_start[i], H_film_start[i],
                H_vapor[i], H_vapor[i],
                H_film_end[i],H_film_end[i],
                H_film_end[i],H_film_end[i]]
        
    else return "Xp error 2"
    end
    
    return X_inner_loop_append,H_inner_loop_append
end

XHloop_append (generic function with 1 method)

In [198]:
AA = rand(1000000);
imin = 500000

500000

In [183]:
@time findmin(AA)

  0.005939 seconds (1 allocation: 32 bytes)


(1.1563798429925498e-7, 569821)

In [184]:
@time argmin(AA)

  0.005433 seconds (1 allocation: 16 bytes)


569821

In [199]:
AA2 = Array{Float64}(undef, 0);

In [204]:
AA2 = []
@time append!(AA2,view(AA, imin:length(AA)))
@time append!(AA2,view(AA, 1:imin-1));

  0.010570 seconds (500.00 k allocations: 11.444 MiB)
  0.010437 seconds (500.00 k allocations: 11.444 MiB)


In [205]:
AA2 = []
@time AA2 = [AA[imin:length(AA)];AA[1:imin-1]]

  0.004764 seconds (10 allocations: 15.259 MiB)


1000000-element Vector{Float64}:
 0.3707465598350228
 0.36875070735466564
 0.7681765956518849
 0.17826840174393488
 0.9569403942156849
 0.08212618723051746
 0.4766527676064414
 0.15320691633526207
 0.5374842036233569
 0.5268734944271081
 0.08443865947175366
 0.8782269367500113
 0.39327431877702246
 ⋮
 0.12553642930120112
 0.9734144371399245
 0.5208508905645763
 0.7406973069622247
 0.013679583047938815
 0.31057043985499644
 0.8201245397363914
 0.5390620547113423
 0.4601038700927724
 0.713119949484941
 0.962484765725782
 0.11578383373457912

In [213]:
AA = rand(10) .- 0.5

10-element Vector{Float64}:
 -0.24517383637157586
  0.19073292719475776
 -0.4617114918937555
 -0.4724946130846719
 -0.13882201771879665
  0.07225190353714339
 -0.08106502956779771
 -0.022247847826107225
 -0.4418463578605569
  0.09088146783871331

In [216]:
max.(AA,0.0)

10-element Vector{Float64}:
 0.0
 0.19073292719475776
 0.0
 0.0
 0.0
 0.07225190353714339
 0.0
 0.0
 0.0
 0.09088146783871331

In [220]:
using interpolation

LoadError: ArgumentError: Package interpolation not found in current path:
- Run `import Pkg; Pkg.add("interpolation")` to install the interpolation package.


In [218]:
quadgk()

LoadError: UndefVarError: quadgk not defined

In [222]:
using Interpolations

In [236]:
interp1

100-element extrapolate(scale(interpolate(::Vector{Float64}, BSpline(Linear())), (1:100,)), Throw()) with element type Float64:
 0.5460267814467406
 0.6645431031987998
 0.8903396091304123
 0.38329031818865844
 0.35110199202647263
 0.9410930801201836
 0.4776995740400889
 0.43361710150315047
 0.4758117806654445
 0.6843683412918968
 0.7339634209309265
 0.806022755665061
 0.5267001380524376
 ⋮
 0.9442547825630421
 0.46403972839049534
 0.16113474153286322
 0.6830377212061058
 0.7754873683583217
 0.3781064374844807
 0.2641464386179906
 0.24624786628906503
 0.1315009204364015
 0.45350959378407585
 0.8676603673483432
 0.34959604455942594

In [237]:
interp2

100-element extrapolate(scale(interpolate(::Vector{Float64}, BSpline(Linear())), (1:100,)), Throw()) with element type Float64:
 0.41567413788914953
 0.9821571883036979
 0.8200669615613498
 0.05392149887683639
 0.028378516617479033
 0.33653156232037107
 0.5717091453551566
 0.3397123906197821
 0.7186559149873271
 0.5721407593886456
 0.6722904462444559
 0.3345441670286997
 0.20697861779580062
 ⋮
 0.31763165793254244
 0.9159198094896048
 0.4048884625988469
 0.9710348429962337
 0.15655313788623193
 0.15391378227976937
 0.26715825627140566
 0.2476806204072146
 0.49610150882444515
 0.7100339041608321
 0.3603786475059989
 0.7071917040531781

In [238]:
interp1 = LinearInterpolation(2:100, rand(99))

interp2 = LinearInterpolation(2:100, rand(99));

In [240]:
interp1 * interp2

LoadError: MethodError: no method matching *(::Interpolations.Extrapolation{Float64, 1, ScaledInterpolation{Float64, 1, Interpolations.BSplineInterpolation{Float64, 1, Vector{Float64}, BSpline{Linear}, Tuple{Base.OneTo{Int64}}}, BSpline{Linear}, Tuple{UnitRange{Int64}}}, BSpline{Linear}, Throw{Nothing}}, ::Interpolations.Extrapolation{Float64, 1, ScaledInterpolation{Float64, 1, Interpolations.BSplineInterpolation{Float64, 1, Vector{Float64}, BSpline{Linear}, Tuple{Base.OneTo{Int64}}}, BSpline{Linear}, Tuple{UnitRange{Int64}}}, BSpline{Linear}, Throw{Nothing}})
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:560
[0m  *([91m::WoodburyMatrices.AbstractWoodbury[39m, ::AbstractVector{T} where T) at /Users/yuxuanli/.julia/packages/WoodburyMatrices/zexu7/src/WoodburyMatrices.jl:50
[0m  *([91m::Dates.Period[39m, ::AbstractArray) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/Dates/src/periods.jl:97
